In [1]:
import pandas as pd
import numpy as np
import glob
import re
from datetime import datetime

from tkinter import *
from tkinter import ttk
from tkinter import filedialog

In [2]:
# Allgemeine Funktionen
def return_error_rows_as_string(error_row):
    error_str = ''
    for e in error_row:
        error_str = error_str + str(e) + ', '
    error_str = error_str[:-2]    
    # Split the string by commas and convert each element to an integer
    numbers_list = [int(num.strip()) for num in error_str.split(',')]
    # Sort the list in ascending order
    numbers_list.sort()
    # Initialize variables
    result = []
    start = numbers_list[0]
    end = numbers_list[0]
    # Iterate over the list and find consecutive numbers
    for i in range(1, len(numbers_list)):
        if numbers_list[i] == end + 1:
            end = numbers_list[i]
        else:
            # Add the range to the result
            if start == end:
                result.append(str(start))
            else:
                result.append(f'{start}-{end}')
            start = numbers_list[i]
            end = numbers_list[i]
    # Add the last range to the result
    if start == end:
        result.append(str(start))
    else:
        result.append(f'{start}-{end}')
    # Join the ranges with commas
    error_str = ', '.join(result)    
    return(error_str)

def open_dm_file(name, path, sheetname):
    """ Öffnet eine der Dateien des DMs.
        Dafür werden der Name der Datei und der Pfad als Argumente übergeben.
    """
    files = glob.glob(path + '\*.xlsx*')
    for i in range(len(files)):
        if name in files[i]:
            path = files[i]
    excel = pd.read_excel(path, sheet_name = sheetname)
    return(excel)

In [23]:
def go_dodo():
    global error_title, root_m
    try:
        error_title.grid_forget()
    except:
        pass
    try:
        root_m.grid_forget()
    except:
        pass
    
    
    vorlage_path = filedialog.askopenfilename(title = "Datei auswählen")
    vorlage = pd.read_csv(vorlage_path, encoding='latin_1', delimiter=';')
    cols = list(vorlage.columns)
    
    ### Interface ###    
    error_title = Label(root, text = 'Error-Meldungen:', bg = '#eeeee4', font=('Ink free',11,'bold'))
    error_title.grid(row=4, column=2, columnspan=5)

    root_m = Frame(root, width=350, height=150, bg = '#eeeee4', highlightbackground='#869287', highlightthickness=2)
    root_m.grid(row=5, column=1, columnspan=10)
    #################
    
    # Einlesen der Excel-Tabelle 'Aufbau und Felder'
    felder = open_dm_file('IDA_Aufbau und Felde', 'O:\Datenmanagement\IDA_in.vent Datenbank', 'Felderbezeichnung')
    felder.columns = felder.iloc[6, :]
    felder = felder.drop(felder.index[:7]).reset_index().drop('index', axis=1)
    # Filtern nach Pflichtfeld:
    # Zeilen rausnehmen, wenn Feld 'nie genutzt' oder 'derzeit nicht genutzt' wird
    felder_not_used = felder[felder['Pflichtfeld'].isin(['nie genutzt', 'derzeit nicht genutzt'])]
    felder = felder[~felder['Feldcode'].isin(felder_not_used['Feldcode'])].reset_index().drop('index', axis = 1)
    
    # Spalten der hochgeladenen Datei mit den Feldern abgleichen und ein Subset erstellen
    index_felder = []
    for col in cols:
        for i in range(len(felder)):
            if col == felder['Feldcode'][i]:
                index_felder.append(i)
    felder_f = felder.iloc[index_felder].reset_index().drop('index', axis=1)


    # Format von pat_id, master_id und sample_id überprüfen
    master_id_pattern = r'I\d{9}'
    sample_id_pattern = r'I\d{12}'

    # Spalte master_id
    if 'master_id' in vorlage.columns:
        error_master_id_row = []
        for i in range(len(vorlage['master_id'])):
            match = re.search(master_id_pattern, vorlage['master_id'][i])
            if match:
                continue
            else:
                error_master_id_row.append(i+2)
        if error_master_id_row != []:
            error_str = return_error_rows_as_string(error_master_id_row)
            text_master = Label(root_m, text = "Fehler in Spalte 'master_id' Zeile(n) {}".format(error_str),
                               bg = '#eeeee4', font=('Ink free',11))
            text_master.pack()

    # Spalte sample_id
    if 'sample_id' in vorlage.columns:
        error_sample_id_row = []
        for i in range(len(vorlage['sample_id'])):
            match = re.search(sample_id_pattern, vorlage['sample_id'][i])
            if match:
                continue
            else:
                error_sample_id_row.append(i+2)
        if error_sample_id_row != []:
            error_str = return_error_rows_as_string(error_sample_id_row)
            text_sample = Label(root_m, "Fehler in Spalte 'sample_id' Zeile(n) {}".format(error_str),
                               bg = '#eeeee4', font=('Ink free',11))
            text_sample.pack()

    # Spalte pat_id
    if 'pat_id' in vorlage.columns:
        error_pat_id_row = []
        # Zuerst wird geprüft, ob die gesamte Spalte im Format np.int64 ist
        if vorlage['pat_id'].dtype != np.int64:
            if vorlage['pat_id'].dtype == 'object':
                for i in range(len(vorlage['pat_id'])):
                    if not str(vorlage['pat_id'][i]).isdigit():
                        error_pat_id_row.append(i+2)
        if error_pat_id_row != []:
            error_str = return_error_rows_as_string(error_pat_id_row)
            text_pat = Label(root_m, text = "(Format-)Fehler in Spalte 'pat_id' Zeile(n) {}".format(error_str),
                            bg = '#eeeee4', font=('Ink free',11))
            text_pat.pack()


    # Überprüfen der Auswahlmöglichkeiten, wenn Feldart monovalent list ist
    for i in range(len(felder_f)):
        field_error_row = []
        if 'Monovalent list' in felder_f['Feldart'][i]:
            check_col = felder_f['Feldcode'][i]
            # falls Auswahlmöglichkeiten in einer weiteren Excel-Datei hinterlegt sind,
            # wird der Pfad angegeben
            possible_choices = felder_f['Auswahlmöglichkeiten bei Listen'][i]
            if 'O:' in possible_choices:
                if '\n' in possible_choices:
                    possible_choices = possible_choices.replace('\n', '')
                path = possible_choices.split('siehe ')[-1]
                if '-->' in path:
                    filename = path.split('-->')[1].strip()
                    path = path.split('-->')[0].strip()
                else:
                    filename = path.split('\\')[-1].split('_')[0]
                    path = '\\'.join(path.split('\\')[:-1])
                files = glob.glob(path+'\*.xlsx')
                for file in files:
                    if (filename in file) | (' '.join(filename.split('_')) in file):
                        path = file                
                df_choices = pd.read_excel(path)
                # erste Spalte ablesen und als Liste abspeichern
                possible_choices = df_choices.iloc[:,0].tolist()
            else:
                if '\n' in possible_choices:
                    possible_choices = possible_choices.split('\n')
                elif ',' in possible_choices:
                    possible_choices = possible_choices.split(',')
            # muss ggf. in List umgewandelt werden
            if not isinstance(possible_choices, list):
                possible_choices = [possible_choices]    
            for j in range(len(possible_choices)):
                c = possible_choices[j].split('=')[0]
                try:
                    c = int(c)
                    possible_choices[j] = possible_choices[j].split('=')[0]
                except:
                    possible_choices[j] = possible_choices[j].split('=')[1]

            possible_choices = [str(int(c)) for c in possible_choices]
            # Ablesen des Eintrags in der zu importierenden Datei (vorlage)
            check_entry = list(vorlage[check_col])
            for j in range(len(check_entry)):
                if np.isnan(check_entry[j]):
                    continue
                else:
                    if str(int(check_entry[j])) not in possible_choices:
                        field_error_row.append(j+2)
        # Falls Fehler in eins der Felder, dann erscheint Fehlermeldung
        if field_error_row != []:
            error_str = return_error_rows_as_string(field_error_row)
            text_field = Label(root_m, text = "Fehler in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                              bg = '#eeeee4', font=('Ink free',11))
            text_field.pack()

    # Datum-Spalte überprüfen (dd.mm.yyyy)
    for i in range(len(felder_f)):
        date_error_row = []
        if 'Date' in felder_f['Feldart'][i]:
            date_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                if str(vorlage[date_col][j]) != 'nan':
                    try:
                        date_string = vorlage[date_col][j]
                        try:
                            date_object = datetime.strptime(date_string, "%d.%m.%Y").date()
                        except:
                            date_object = datetime.strptime(date_string, "%Y-%m").date()
                    except ValueError:
                        date_error_row.append(j+2)
        if date_error_row != []:
            error_str = return_error_rows_as_string(date_error_row)
            text_date = Label(root_m, text = "Fehler in Datumspalte '{}' Zeile(n) {}".format(date_col, error_str),
                             bg = '#eeeee4', font=('Ink free',11))
            text_date.pack()

    # 'One line text'-Felder überprüfen, sodass es eine Warnung gibt,
    # wenn Text ab dem 120. Zeichen abgeschnitten wird
    for i in range(len(felder_f)):
        long_text_row = []
        if 'One line text' in felder_f['Feldart'][i]:
            check_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                text = vorlage[check_col][j]
                if len(str(text)) > 120:
                    long_text_row.append(j+2)
        if long_text_row != []:
            error_str = return_error_rows_as_string(long_text_row)
            text_long = Label(root_m, text = "Text wurde abgeschnitten in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                             bg = '#eeeee4', font=('Ink free',11))
            text_long.pack()


    # Katalog-Felder überprüfen
    for i in range(len(felder_f)):
        field_error_row = []
        if 'Katalog' in felder_f['Feldart'][i]:
            check_col = felder_f['Feldcode'][i]
            possible_choices = felder_f['Auswahlmöglichkeiten bei Listen'][i]
            if '\n' in possible_choices:
                possible_choices = possible_choices.replace('\n', '')
            if 'O:' in possible_choices:
                path = possible_choices.split('siehe ')[-1]
                if '-->' in path:
                    filename = path.split('-->')[1].strip()
                    path = path.split('-->')[0].strip()
                else:
                    filename = path.split('\\')[-1].split('_')[0]
                    path = '\\'.join(path.split('\\')[:-1])
                files = glob.glob(path+'\*.xlsx')
                for file in files:
                    if (filename in file) | (' '.join(filename.split('_')) in file):
                        path = file                
            df_choices = pd.read_excel(path)
            # erste Spalte ablesen und als Liste abspeichern
            possible_choices = df_choices.iloc[:,0].tolist()
            possible_choices = [str(int(c)) for c in possible_choices if str(c) != 'nan']
            # Ablesen des Eintrags in der zu importierenden Datei (vorlage)
            check_entry = list(vorlage[check_col])
            for j in range(len(check_entry)):
                if np.isnan(check_entry[j]):
                    continue
                else:
                    if str(int(check_entry[j])) not in possible_choices:
                        field_error_row.append(j+2)
        # Falls Fehler in eins der Felder, dann erscheint Fehlermeldung
        if field_error_row != []:
            error_str = return_error_rows_as_string(field_error_row)
            text_kat = Label(root_m, text = "Fehler in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                            bg = '#eeeee4', font=('Ink free',11))
            text_kat.pack()



    # Feldcode-Namen überprüfen
    for col in cols:
        if col not in list(felder_f['Feldcode']):
            text_feldcode = Label(root_m, text = "Bitte Feldcode '{}' überprüfen".format(col),
                                 bg = '#eeeee4', font=('Ink free',11))
            text_feldcode.pack()

    # Pflichtfelder füllen
    for i in range(len(felder_f)):
        pflichtfeld_row = []
        soll_row = []
        if (felder_f['Pflichtfeld'][i] == 'Pflichtfeld') | ('Pflichtfeld?' in felder_f['Pflichtfeld'][i]) | ('leeres Pflichtfeld' in felder_f['Pflichtfeld'][i]):
            check_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                if str(vorlage[check_col][j]) == 'nan':
                    pflichtfeld_row.append(j+2)
            if pflichtfeld_row != []:
                error_str = return_error_rows_as_string(pflichtfeld_row)
                text_pflicht = Label(root_m, text = "Pflichtfeld ist leer in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                                    bg = '#eeeee4', font=('Ink free',11))
                text_pflicht.pack()
        elif ('soll' in felder_f['Pflichtfeld'][i]) & ('Angabe' not in felder_f['Pflichtfeld'][i]):
            check_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                if str(vorlage[check_col][j]) == 'nan':
                    soll_row.append(j+2)
            if soll_row != []:
                error_str = return_error_rows_as_string(soll_row)
                text_soll = Label(root_m, text = "Bitte Feld füllen in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                                  bg = '#eeeee4', font=('Ink free',11))
                text_soll.pack()

In [25]:
# User Interface
# Basis
root = Tk()
root.title('Import-Check')
root.geometry('500x350')
root.iconbitmap('O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo_icon.ico')
root.config(bg='#eeeee4')

# alle Buttons auf Startseite
hi = Label(root, text = '\nÜberprüfen der Dateien\nvor dem Import in IDA\n', bg = '#eeeee4', font=('Ink free',15,'bold'))
hi.grid(row=1, column=1, columnspan=10)

browse_text = Label(root, text = 'Bitte wähle die Datei aus.  ', bg = '#eeeee4', font=('Ink free',12))
browse_text.grid(row=2, column=1, columnspan=5)
browse_button = Button(root, text='Browse', font=('Ink free',10,'bold'), bg='#869287',
                       command = go_dodo)
browse_button.grid(row=2, column=6)
space = Label(root, text = '', bg = '#eeeee4', font=('Ink free',12))
space.grid(row=3, column=1)



# Logo
from PIL import ImageTk, Image
frame = Frame(root, width=1, height=1)
frame.grid(row=1, column=0)

img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((70,70)), master = root)
label = Label(frame, image = img, bg = '#eeeee4')
label.pack()

root.mainloop()